# CPLEX Solver

In [1]:
import cplex
from collections import namedtuple

In [2]:
SubjectGroup = namedtuple('SubjectGroup', ('input_capacity', 'output_capacity', 'area'))

In [3]:
Point = namedtuple('Point', ('x', 'y', 'area'))

In [4]:
class TotalFlows:
	def __init__(self, groups: "dict[str, SubjectGroup]"):
		self.total_flow = {(i, j) : 0 for i in groups for j in groups}
		self.groups = tuple(groups.keys())
		return

	def set_flow(self, group_a: str, group_b: str, new_flow: int):
		if (group_a, group_b) not in self.total_flow:
			raise KeyError
		self.total_flow[(group_a, group_b)] = new_flow
		return
	
	def __getitem__(self, key: "tuple[str, str]"):
		return self.total_flow[key]
	
	def get_in_flow(self, group: str):
		return sum(self.total_flow[(i, group)] for i in self.groups)
	
	def get_out_flow(self, group: str):
		return sum(self.total_flow[(group, j)] for j in self.groups)

In [5]:
def arrange_0(points: "dict[str, Point]", distance, groups: "dict[str, SubjectGroup]", total_flows: TotalFlows):
	ceil = lambda x: x if x == int(x) else int(x) + 1
	# Compute the sufficient number of subjects for each group
	total_subject_count = {i : max(ceil(total_flows.get_in_flow(i) / groups[i].input_capacity), ceil(total_flows.get_out_flow(i) / groups[i].output_capacity)) for i in groups}
	# Compute total production by each subject group
	total_production_count = {i : total_flows.get_out_flow(i) - total_flows.get_in_flow(i) for i in groups}
	# CPLEX model
	cplex_model = cplex.Cplex()
	cplex_model.objective.set_sense(cplex_model.objective.sense.minimize)
	# CPLEX variables
	cplex_model.variables.add \
	(
		names=[f'f({i},{j})[{u},{v}]' for i in groups for j in groups for u in points for v in points],
		lb=[0] * len(groups)**2 * len(points)**2,
		ub=[total_flows[(i, j)] for i in groups for j in groups for u in points for v in points],
		types=[cplex_model.variables.type.integer] * len(groups)**2 * len(points)**2
	)
	cplex_model.variables.add \
	(
		names=[f'n({i})[{u}]' for i in groups for u in points],
		lb=[0] * len(groups) * len(points),
		ub=[total_subject_count[i] for i in groups for u in points],
		types=[cplex_model.variables.type.integer] * len(groups) * len(points)
	)
	cplex_model.variables.add \
	(
		names=[f'g({i})[{u}]' for i in groups for u in points],
		lb=[0 if total_production_count[i] >= 0 else total_production_count[i] for i in groups for u in points],
		ub=[total_production_count[i] if total_production_count[i] >= 0 else 0 for i in groups for u in points],
		types=[cplex_model.variables.type.integer] * len(groups) * len(points)
	)
	# Objective function
	cplex_model.objective.set_linear(tuple((f'f({i},{j})[{u},{v}]', distance[(u, v)]) for i in groups for j in groups for u in points for v in points))
	# Constraints
	for i in groups:
		for u in points:
			cplex_constr_output = cplex.SparsePair \
			(
				ind=tuple(f'f({i},{j})[{u},{v}]' for j in groups for v in points) + (f'n({i})[{u}]',),
				val=(1,) * len(groups) * len(points)                              + (- groups[i].output_capacity,)
			)
			cplex_constr_input = cplex.SparsePair \
			(
				ind=tuple(f'f({j},{i})[{v},{u}]' for j in groups for v in points) + (f'n({i})[{u}]',),
				val=(1,) * len(groups) * len(points)                              + (- groups[i].input_capacity,)
			)
			cplex_constr_kirchhoff = cplex.SparsePair \
			(
				ind=tuple(f'f({i},{j})[{u},{v}]' for j in groups for v in points) + (f'g({i})[{u}]',) + tuple(f'f({j},{i})[{v},{u}]' for j in groups for v in points if j != i),
				val=(1,) * len(groups) * len(points)                              + (- 1,)            + tuple(- 1 for j in groups for v in points if j != i)
			)
			cplex_model.linear_constraints.add \
			(
				lin_expr=(cplex_constr_output, cplex_constr_input, cplex_constr_kirchhoff),
				senses=('L', 'L', 'E'),
				rhs=(0, 0, 0)
			)
		cplex_constr_total_production = cplex.SparsePair \
		(
			ind=tuple(f'g({i})[{u}]' for u in points),
			val=(1,) * len(points)
		)
		cplex_constr_total_subject_count = cplex.SparsePair \
		(
			ind=tuple(f'n({i})[{u}]' for u in points),
			val=(1,) * len(points)
		)
		cplex_model.linear_constraints.add \
		(
			lin_expr=(cplex_constr_total_production, cplex_constr_total_subject_count),
			senses=('E', 'E'),
			rhs=(total_production_count[i], total_subject_count[i])
		)
		for j in groups:
			cplex_constr_total_flow = cplex.SparsePair \
			(
				ind=tuple(f'f({i},{j})[{u},{v}]' for u in points for v in points),
				val=(1,) * len(points)**2
			)
			cplex_model.linear_constraints.add \
			(
				lin_expr=(cplex_constr_total_flow,),
				senses=('E',),
				rhs=(total_flows[(i, j)],)
			)
	for u in points:
		cplex_constr_area = cplex.SparsePair \
		(
			ind=tuple(f'n({i})[{u}]' for i in groups),
			val=tuple(groups[i].area for i in groups)
		)
		cplex_model.linear_constraints.add \
		(
			lin_expr=(cplex_constr_area,),
			senses=('L',),
			rhs=(points[u].area,)
		)
	# Solve
	cplex_model.solve()
	if (cplex_model.solution.get_status() != 103):
		cplex_model.solution.write('test.sol')
	return

## 2. Input data

In [6]:
points = {f'({i},{j})' : Point(i, j, 1) for i in range(1, 4) for j in range(1, 4)}
groups = {'A' : SubjectGroup(1, 30, 1), 'B' : SubjectGroup(25, 20, 1), 'C' : SubjectGroup(100, 1, 1)}
distance = {(u, v) : abs(points[u].x - points[v].x) + abs(points[u].y - points[v].y) for u in points for v in points}
total_flows = TotalFlows(groups)
total_flows.set_flow('A', 'B', 55)
total_flows.set_flow('A', 'C', 45)
total_flows.set_flow('B', 'C', 40)

In [7]:
arrange_0(points, distance, groups, total_flows)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 24 rows and 486 columns.
MIP Presolve modified 324 coefficients.
Aggregator did 18 substitutions.
Reduced MIP has 61 rows, 279 columns, and 999 nonzeros.
Reduced MIP has 27 binaries, 252 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.29 ticks)
Found incumbent of value 215.000000 after 0.00 sec. (2.17 ticks)
Probing fixed 0 vars, tightened 27 bounds.
Probing time = 0.01 sec. (0.29 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 0 rows and 27 columns.
Reduced MIP has 61 rows, 252 columns, and 900 nonzeros.
Reduced MIP has 27 binaries, 225 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.66 ticks)
Probing time = 0.00 sec. (0.27 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic